In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import sys
import time

from matplotlib import pyplot as plt
from dotenv import load_dotenv

sys.path.append("../")

data_path = "../data"

load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100

start = time.perf_counter()


In [ ]:
csv_list = []

for dirpath, subdirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".csv") and dirpath != "../data/optimized":
            csv_list.append(file)

df_dict = {}

for csv in csv_list:
    file_name = f"../data/{csv}"
    df_dict[f"{csv[:-4]}"] = pd.read_csv(file_name)

# Our dfs :

for key in df_dict:
    print(key)


# 1 : Customers

In [ ]:
df_customers = df_dict["olist_customers_dataset"]

df_customers.head()


## 1.1 : Aliases (customer_id)

> Replacing aliases and enforcing new dtype

--> Datasets containing "customer_id" are :
- olist_customers_dataset
- olist_orders_dataset


In [ ]:
customer_aliases = df_customers["customer_id"].unique()

# lenght of df should be len of list of unique aliases are aliases are single use. Checking

print(len(df_customers) == len(customer_aliases))


In [ ]:
aliases_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for alias in customer_aliases:
    aliases_rekey[alias] = current_id
    current_id += 1


In [ ]:
# Checking that first and last are in dict :

print(df_customers.iloc[0]["customer_id"] in aliases_rekey.keys())
print(df_customers.iloc[-1]["customer_id"] in aliases_rekey.keys())


### 1.1.1 : In Customers


In [ ]:
df_customers_light = df_customers.copy()

df_customers_light.head()


In [ ]:
df_customers_light["customer_id"].replace(to_replace=aliases_rekey, inplace=True)


### 1.1.2 : In orders

In [ ]:
df_orders = df_dict["olist_orders_dataset"]

df_orders_light = df_orders.copy()

df_orders_light.head()


In [ ]:
df_orders_light["customer_id"].replace(to_replace=aliases_rekey, inplace=True)


In [ ]:
df_customers_light["customer_id"] = df_customers_light["customer_id"].astype(np.uint32)
df_orders_light["customer_id"] = df_orders_light["customer_id"].astype(np.uint32)


## 1.2 : Customers_unique_id

- Only in customers table

In [ ]:
customers_uids = df_customers["customer_unique_id"].unique()  # Unique id is not unique in this DB


In [ ]:
uids_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for uid in customers_uids:
    uids_rekey[uid] = current_id
    current_id += 1


In [ ]:
df_customers_light["customer_unique_id"].replace(to_replace=uids_rekey, inplace=True)


In [ ]:
df_customers_light["customer_unique_id"] = df_customers_light["customer_unique_id"].astype(np.uint32)


In [ ]:
df_customers_light.info()


# 2 : Order_id

> Replacing hexa order ids and enforcing new dtype (uint32)

--> Datasets containing "order_id" are :
- olist_orders_dataset --> already used as "df_orders_light | df_orders_light" 
- olist_order_items_dataset --> df_order_items | df_order_items_light (creating)
- olist_order_payments_dataset --> df_payments | df_payments_light (creating)
- olist_order_reviews_dataset --> df_reviews | df_reviews_light (creating)


In [ ]:
df_order_items = df_dict["olist_order_items_dataset"]

df_order_items_light = df_order_items.copy()

df_order_items_light.head()


In [ ]:
df_payments = df_dict["olist_order_payments_dataset"]

df_payments_light = df_payments.copy()

df_payments_light.head()


In [ ]:
df_reviews = df_dict["olist_order_reviews_dataset"]

df_reviews_light = df_reviews.copy()

df_reviews_light.head()


#### 2.0 Collecting all order_ids and ensuring that all are present

> selection through cross checking with others dfs

In [ ]:
order_id_list = df_orders["order_id"].unique()


In [ ]:
order_id_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for order_id in order_id_list:
    order_id_rekey[order_id] = current_id
    current_id += 1


## 2.1 : Modifying order_ids and enforcing dtype = uint32

In [ ]:
df_orders_light["order_id"].replace(to_replace=order_id_rekey, inplace=True)


In [ ]:
df_order_items_light["order_id"].replace(to_replace=order_id_rekey, inplace=True)


In [ ]:
df_payments_light["order_id"].replace(to_replace=order_id_rekey, inplace=True)


In [ ]:
df_reviews_light["order_id"].replace(to_replace=order_id_rekey, inplace=True)


In [ ]:
df_orders_light["order_id"] = df_orders_light["order_id"].astype(np.uint32)
df_order_items_light["order_id"] = df_order_items_light["order_id"].astype(np.uint32)
df_payments_light["order_id"] = df_payments_light["order_id"].astype(np.uint32)
df_reviews_light["order_id"] = df_reviews_light["order_id"].astype(np.uint32)


# 3 : Product_id

> datasets using product_id : 
- olist_products_dataset --> df_products | df_products_light
- olist_order_items_dataset --> already used as "df_order_items | df_orders_items_light" 

In [ ]:
df_products = df_dict["olist_products_dataset"]

df_products_light = df_products.copy()

df_products_light.head()


In [ ]:
products_id_list = df_products["product_id"].unique()


In [ ]:
product_id_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for product_id in products_id_list:
    product_id_rekey[product_id] = current_id
    current_id += 1


In [ ]:
df_products_light["product_id"].replace(to_replace=product_id_rekey, inplace=True)


In [ ]:
df_order_items_light["product_id"].replace(to_replace=product_id_rekey, inplace=True)


In [ ]:
df_products_light["product_id"] = df_products_light["product_id"].astype(np.uint32)
df_order_items_light["product_id"] = df_order_items_light["product_id"].astype(np.uint32)


# 4 : Sellers Id

> sellers_id in :
- olist_sellers_dataset --> df_sellers | df_sellers_light (creating)
- olist_order_items_dataset --> (already used) df_order_items | df_order_items_light

In [ ]:
df_sellers = df_dict["olist_sellers_dataset"]

df_sellers_light = df_sellers.copy()

df_sellers_light.head()


In [ ]:
sellers_id_list = df_sellers_light["seller_id"].unique()


In [ ]:
# Checking all known sellers from order_items are in sellers_id

print(all(item in df_order_items["seller_id"].values for item in sellers_id_list))


In [ ]:
seller_id_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for seller_id in sellers_id_list:
    seller_id_rekey[seller_id] = current_id
    current_id += 1


In [ ]:
df_order_items_light["seller_id"].replace(to_replace=seller_id_rekey, inplace=True)


In [ ]:
df_sellers_light["seller_id"].replace(to_replace=seller_id_rekey, inplace=True)


In [ ]:
df_order_items_light["seller_id"] = df_order_items_light["seller_id"].astype(np.uint32)
df_sellers_light["seller_id"] = df_sellers_light["seller_id"].astype(np.uint32)


In [ ]:
df_dict.keys()


# 5 : Optimisation of all datasets, comparison & exports

## 5.1 : olist_sellers_dataset

--> Infos :

In [ ]:
df_sellers_light.info()

In [ ]:
df_sellers_light.head()

#### Optimizations :

- Zip Code to optimal int
- city & state --> No change (to pandas, str = object)


In [ ]:
df_sellers_light["seller_zip_code_prefix"].values.max()


In [ ]:
# 99730 is over uint 16 max value (65K and changes), uint 32 is less heavy
df_sellers_light["seller_zip_code_prefix"] = df_sellers_light["seller_zip_code_prefix"].astype(np.uint32)


In [ ]:
df_sellers_light.info()

In [ ]:
df_sellers_light.to_csv(path_or_buf="../data/optimized/olist_sellers.csv", index=False)
df_sellers_light.to_pickle(path="../pickles/olist_sellers.pkl")


# 5.2 : olist_orders_dataset

--> Infos

In [ ]:
df_orders_light.info()


In [ ]:
df_orders_light.head()


#### Optimizations :
- Rename "order_purchase_timestamp" to "order_purchase_date" to reflect format
- Type enforcing dates for "order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date" "order_delivered_customer_date", "order_estimated_delivery_date" --> type = Datetime

In [ ]:
df_orders_light.rename(columns={"order_purchase_timestamp": "order_purchase_dt"}, inplace=True)


In [ ]:
date_cols = [
        "order_purchase_dt", "order_approved_at",
        "order_delivered_carrier_date", "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ]

for col in date_cols:
    df_orders_light[col] = pd.to_datetime(df_orders_light[col])


In [ ]:
df_orders_light.info()


In [ ]:
df_orders_light.to_csv(path_or_buf="../data/optimized/olist_orders.csv", index=False)
df_orders_light.to_pickle(path="../pickles/olist_orders.pkl")


# 5.3 : olist_order_items_dataset

--> Infos

In [ ]:
df_order_items_light.info()


In [ ]:
df_order_items_light.head()


#### Optimizations :
- order_item_id min_max type
- enforce dtype = datetime on shipping_limit_date

In [ ]:
df_order_items_light["order_item_id"].values.max()


In [ ]:
# 21 is able to go down to uint8

df_order_items_light["order_item_id"] = df_order_items_light["order_item_id"].astype(np.uint8)


In [ ]:
df_order_items_light["shipping_limit_date"] = pd.to_datetime(df_order_items_light["shipping_limit_date"])


In [ ]:
df_order_items_light.info()


In [ ]:
df_order_items_light.to_csv(path_or_buf="../data/optimized/olist_order_items.csv", index=False)
df_order_items_light.to_pickle(path="../pickles/olist_order_items.pkl")


# 5.4 : olist_customers_dataset

--> Infos :

In [ ]:
df_customers_light.info()


In [ ]:
df_customers_light.head()


#### Optimization :

- Zip Code down to uint32

In [ ]:
zip_pre = "customer_zip_code_prefix"
df_customers_light[zip_pre] = df_customers_light[zip_pre].astype(np.uint32)


In [ ]:
df_customers_light.to_csv(path_or_buf="../data/optimized/olist_customers.csv", index=False)
df_customers_light.to_pickle(path="../pickles/olist_customers.pkl")


# 5.5 : olist_order_payments_dataset

--> Infos :

In [ ]:
df_payments_light.info()


In [ ]:
df_payments_light.head()


#### Optimizations : 

- MinMax "payment_sequential" & "payment_installments" dtypes


In [ ]:
print("max payment_sequential : ", df_payments_light["payment_sequential"].values.max())
print("max payment_installments : ", df_payments_light["payment_installments"].values.max())

In [ ]:
df_payments_light["payment_sequential"] = df_payments_light["payment_sequential"].astype(np.uint8)
df_payments_light["payment_installments"] = df_payments_light["payment_installments"].astype(np.uint8)


In [ ]:
df_payments_light.info()


In [ ]:
df_payments_light.to_csv(path_or_buf="../data/optimized/olist_payments.csv", index=False)
df_payments_light.to_pickle(path="../pickles/olist_payments.pkl")


# 5.6 : olist_order_reviews_dataset

--> Infos :

In [ ]:
df_reviews_light.info()


In [ ]:
df_reviews_light.head()


#### Optimization :

- Review id from 32hexa to int
- review score min_max
- review_answer_timestamp rename to review_answer_dt
- enforcing correct dtype for both datetimes

In [ ]:
review_id_list = df_reviews_light["review_id"].unique()

print("Same len : ", len(review_id_list) == len(df_reviews_light))


In [ ]:
review_id_rekey = {}

current_id = 1  # Easier for R/Julia/Etc.

for review_id in review_id_list:
    review_id_rekey[review_id] = current_id
    current_id += 1


In [ ]:
df_reviews_light["review_id"].replace(to_replace=review_id_rekey, inplace=True)


In [ ]:
df_reviews_light["review_score"].values.max()


In [ ]:
df_reviews_light.rename(columns={"review_answer_timestamp": "review_answer_dt"}, inplace=True)

df_reviews_light["review_answer_dt"] = pd.to_datetime(df_reviews_light["review_answer_dt"])
df_reviews_light["review_creation_date"] = pd.to_datetime(df_reviews_light["review_creation_date"])


In [ ]:
df_reviews_light["review_score"] = df_reviews_light["review_score"].astype(np.uint8)
df_reviews_light["review_id"] = df_reviews_light["review_id"].astype(np.uint32)


In [ ]:
df_reviews_light.info()


In [ ]:
df_reviews_light.to_csv(path_or_buf="../data/optimized/olist_reviews.csv", index=False)
df_reviews_light.to_pickle(path="../pickles/olist_reviews.pkl")


# 5.7 : olist_products_dataset

--> Infos :

In [ ]:
df_products_light.info()


In [ ]:
df_products_light.head()


#### Optimizations :

- product_name_lenght to minimum int (see below *)
- product_description_lenght to minimum int (counting chars does not require floating point)
- product_photos_qty to int (see above)

In [ ]:
print("max name length : ", max(df_products_light["product_name_lenght"].values))
print("max desc. length : ", max(df_products_light["product_description_lenght"].values))
print("max photo qty : ", max(df_products_light["product_photos_qty"].values))


In [ ]:
# unsigned 8, 16 and 8 respectively : 

# Nan count :

print("Product Name Lenght NAN : ", df_products_light["product_name_lenght"].isna().sum())
print("Product Desc Lenght NAN : ", df_products_light["product_description_lenght"].isna().sum())
print("Product Photo Qty. NAN : ", df_products_light["product_photos_qty"].isna().sum())


In [ ]:
# Fill w/ 0s

df_products_light["product_name_lenght"].fillna(value=0, inplace=True)
df_products_light["product_description_lenght"].fillna(value=0, inplace=True)
df_products_light["product_photos_qty"].fillna(value=0, inplace=True)


In [ ]:
# unsigned 8, 16 and 8 respectively : 

print("Product Name Lenght NAN : ", df_products_light["product_name_lenght"].isna().sum())
print("Product Desc Lenght NAN : ", df_products_light["product_description_lenght"].isna().sum())
print("Product Photo Qty. NAN : ", df_products_light["product_photos_qty"].isna().sum())


In [ ]:
df_products_light["product_name_lenght"] = df_products_light["product_name_lenght"].astype(np.uint8)
df_products_light["product_description_lenght"] = df_products_light["product_description_lenght"].astype(np.uint16)
df_products_light["product_photos_qty"] = df_products_light["product_photos_qty"].astype(np.uint8)


In [ ]:
df_products_light.info()


In [ ]:
df_products_light.to_csv(path_or_buf="../data/optimized/olist_products.csv", index=False)
df_products_light.to_pickle(path="../pickles/olist_products.pkl")


# Conclusion : 

- The goal of this notebook was purely to optimize future operations by enforcing correct data types, removing hexa 32 chars for a lighter format (uint32) as ids
- A huge part could (and would, if not for a few blue screens) optimized by using as multiprocessing Pool on pd.DataFrame.replace() operatations. Most computers designed for those tasks have at least 8 cores so we could expect a minimum of around 8times faster exec with mp.
- Let's calculate the raw gain of memory, ignoring any gain resulting from a comparison etc., just the rough original vs. light diff.

In [ ]:
# Using DataFrame.memory_usage(index=True).sum()

diff_dict = {}

diff_dict["sellers"] = df_sellers.memory_usage(index=True).sum() - df_sellers_light.memory_usage(index=True).sum()
diff_dict["orders"] = df_orders.memory_usage(index=True).sum() - df_orders_light.memory_usage(index=True).sum()
diff_dict["order_items"] = df_order_items.memory_usage(index=True).sum() - df_order_items_light.memory_usage(index=True).sum()
diff_dict["customers"] = df_customers.memory_usage(index=True).sum() - df_customers_light.memory_usage(index=True).sum()
diff_dict["payments"] = df_payments.memory_usage(index=True).sum() - df_payments_light.memory_usage(index=True).sum()
diff_dict["reviews"] = df_reviews.memory_usage(index=True).sum() - df_reviews_light.memory_usage(index=True).sum()
diff_dict["products"] = df_products.memory_usage(index=True).sum() - df_products_light.memory_usage(index=True).sum()


In [ ]:
diff_dict

In [ ]:
total_diff = sum(list(diff_dict.values()))

print(f"total diff = {total_diff / 1000000} Mega bytes")


8.3Mbytes, not bad, close to two MP3 songs in optimization

In [ ]:
og_size = 0

for df in df_dict:
    og_size += df_dict[df].memory_usage(index=True).sum()

print(og_size / 1000000)


In [ ]:
end = time.perf_counter()

execution_period = (end - start)
print(f"Duration of execution {execution_period}s")

print(f"Reduction of {round((total_diff / og_size) * 100, ndigits=2)}%")


It takes a while but it will save us more that it cost us to convert. The type enforcement will make calculations faster and we shaved around 29% of the original dataset.